In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json
import urllib

In [2]:
# s = "https://yandex.ru/maps/239/sochi/stops/stop__9973362/"
city_code = '239'
city_name = 'sochi'
stop = 'stop__9973362'

s0 = 'href="/maps/'+str(city_code)+'/'+city_name+'/routes/'

s = 'https://yandex.ru/maps/{}/{}/stops/{}/'.format(city_code, city_name, stop)

In [3]:
url = s

In [4]:
req = requests.get(url)
bs = BeautifulSoup(req.text, 'html.parser')

In [5]:
lst_links = bs.find_all('a')

In [6]:
lst_routes_links = []
route_names = []
titles = []
for link in lst_links:
    if s0 in str(link) and link.text != 'Маршруты':
        lst_routes_links.append('https://yandex.ru'+link.get('href'))
        route_names.append(link.text)
        titles.append(link.get('title'))
# 

In [7]:
url1 = lst_routes_links[0]

In [8]:
req = requests.get(url1)
bs = BeautifulSoup(req.text, 'html.parser')

In [9]:
scrpt = bs.find_all('script')[-2].text
js = json.loads(scrpt)

In [10]:
js['masstransitLine']['features'][0]['features'][-1]

{'coordinates': [39.773594992, 43.557614811],
 'id': 'stop__9973554',
 'name': 'Микрорайон Искра',
 'region': {'bounds': [[39.1261495, 43.261256799670214],
   [40.3199745, 43.908055799670215]],
  'hierarchy': [225, 10995, 239],
  'id': 239,
  'seoname': 'sochi'}}

In [11]:
bs.find_all('script')[-2].get("type") == "application/json"

True

In [14]:
all_scrpt = bs.find_all('script')
for scr in all_scrpt:
    if scr.get("type") == "application/json":
        print(all_scrpt.index(scr))
# 

12


In [16]:
str1 = 'остановка общественного транспорта'
str_qt = urllib.parse.quote(str1)

In [20]:
srch = "https://yandex.ru/maps/239/sochi/search/{}".format(str_qt)

In [21]:
req = requests.get(srch)
bs = BeautifulSoup(req.text, 'html.parser')

In [33]:
all_scrpt = bs.find_all("script")

In [34]:
all_scrpt = bs.find_all("script")
for scr in all_scrpt:
    if scr.get('type') == "application/json":
        print(all_scrpt.index(scr))

14


In [28]:
scr1 = all_scrpt[13].text
js2 = json.loads(scr1)

In [32]:
js2.keys()

dict_keys(['counters', 'hosts', 'locale', 'lang', 'tld', 'query', 'baseUrl', 'originalUrl', 'uatraits', 'cspNonce', 'origin', 'csrfToken', 'xscriptCsrfToken', 'isUserFromPortal', 'isUserFromGoogle', 'availableLangs', 'currencyCode', 'banner', 'staticHost', 'pageMode', 'bookmarks', 'clientErrors', 'searchResultsCount', 'enableTaxi', 'enableAvia', 'enableZapravkiPromo', 'iframeCounter', 'tnsCounter', 'cloudApi', 'blaBlaCarBanner', 'search', 'routes', 'feedback', 'font', 'advert', 'serviceName', 'bilet', 'share', 'mobile', 'servicePanel', 'apiManual', 'mapMargin', 'business', 'troubleshooting', 'passport', 'morda', 'ruler', 'ugcCabinet', 'advertLandingUrl', 'advertLandingMetroUrl', 'becomeAuthorUrl', 'becomeOwnerUrl', 'agreementUrl', 'metroAgreementUrl', 'rightHoldersUrl', 'ymapsUrlTemplate', 'mrcLandingUrl', 'reviews', 'enableRoutesB2B', 'routesB2BUrl', 'enableMobileDistribution', 'bannerLayout', 'pharmacyLinks', 'mapLocation', 'mapType', 'userRegion', 'experiments', 'layerLinks', 'layer

In [34]:
js2['searchPreloadedResults'].keys()

dict_keys(['requestId', 'requestSerpId', 'requestContext', 'requestQuery', 'requestCorrectedQuery', 'requestBounds', 'displayType', 'totalResultCount', 'requestResults', 'requestSkip', 'items', 'bounds', 'categories', 'filters', 'hasLavkaBanner'])

In [48]:
js2['searchPreloadedResults']['items'][1].keys()

dict_keys(['type', 'requestId', 'analyticsId', 'title', 'description', 'address', 'coordinates', 'displayCoordinates', 'bounds', 'logId', 'uri', 'id', 'metro', 'stops', 'panorama', 'shortTitle', 'fullAddress', 'country', 'status', 'businessLinks', 'ratingData', 'sources', 'categories', 'businessProperties', 'modularSnippet', 'seoname', 'geoId', 'compositeAddress', 'references', 'subtitleItems', 'region', 'breadcrumbs'])

In [53]:
js2['searchPreloadedResults']['items'][1]['description']

'Россия, Краснодарский край, Сочи, улица Конституции СССР'

# Способ поиска всех маршрутов в городе по названию

In [ ]:
import pandas as pd
import shapely
from shapely.geometry import Point, LineString, MultiLineString

In [80]:
str_ct = 'Самара'
str_qt = urllib.parse.quote(str_ct)
srch_ct = "https://yandex.ru/maps/?text={}".format(str_qt)

In [81]:
req = requests.get(srch_ct)
bs = BeautifulSoup(req.text, 'html.parser')

In [83]:
alla = bs.find_all('a')
for aa in alla:
    cls = aa.get("class")
    try:
        if 'catalog-entry-point' in cls:
            hrf = aa.get("href")
            break
    except:
        pass
# 
try:
    city_id,city_name = hrf.split("maps/")[1].split("/catalog")[0].split("/")
except:
    city_id,city_name = None,None
    print("Error. Not found")
    pass
#


In [85]:
lnk_tp = "https://yandex.ru/maps/"+ str(city_id) + "/" + str(city_name) + "/transport"

In [96]:
def find_js(lnk):
    req = requests.get(lnk)
    bs = BeautifulSoup(req.text, 'html.parser')
    all_scrpt = bs.find_all("script")
    for scr in all_scrpt:
        if scr.get('type') == "application/json":
            sc_ind = all_scrpt.index(scr)
            break
    # 
    scrp_txt = all_scrpt[sc_ind].text
    js = json.loads(scrp_txt)
    
    return js

In [105]:
js_tp = find_js(lnk_tp)

In [ ]:
link

In [379]:
req = requests.get(lnk_tp)
bs = BeautifulSoup(req.text, 'html.parser')
lst_srp = bs.find_all('script')

In [384]:
lst_srp[11]

<script type="application/ld+json">{"@context":"http://schema.org","@type":"BreadcrumbList","itemListElement":[{"@type":"ListItem","position":1,"item":{"@id":"https://yandex.ru/maps/","name":"Карты"}},{"@type":"ListItem","position":2,"item":{"@id":"https://yandex.ru/maps/51/samara/","name":"Самара"}}]}</script>

In [387]:
lst_lineid = []
lst_tpts = []
lst_names = []
lst_rts = []
i=0
for i in range(len(js_tp['vehicles'])):
    one_lst = []
    try:
        meta_veh = js_tp['vehicles'][i]['properties']['VehicleMetaData']['Transport']
        lineid = meta_veh['lineId']
        if lineid not in lst_lineid:
            lst_lineid.append(lineid)
            one_lst.append(lineid)
            answ_line = 1
        else:
            answ_line = 0
        try:
            transp_type = meta_veh['type']
            name_rt = meta_veh['name']
        except:
            transp_type,name_rt = None,None
        if answ_line == 1:
            lst_tpts.append(transp_type)
            lst_names.append(name_rt)
            one_lst.append(name_rt)
            one_lst.append(transp_type)
        #
        if one_lst != []:
            lst_rts.append(one_lst)
    except:
        pass
# 

In [411]:
def get_stop(dct):
    lst=[]
    stop_id = dct['id']
    name = dct['name']
    stop_lon = dct['coordinates'][0]
    stop_lat = dct['coordinates'][1]
    lst = [stop_id,name,stop_lat,stop_lon]
    return lst

In [420]:
def get_dir_data(one_dir,direction,rt_id):
    all_stops=[]
    rt_line=[]
    line_data=[]
    i=0
    lst_stop=[]
    cnt_stp = 0
    line_data = line_data + [rt_id,direction]
    for i in range(len(one_dir)):
        if 'id' in one_dir[i]:
            cnt_stp +=1
            lst_stop = []
            lst_stop = lst_stop + [rt_id,direction,cnt_stp]
            lst_stop = lst_stop + get_stop(one_dir[i])
            all_stops.append(lst_stop)

        if 'points' in one_dir[i]:
            one_part = one_dir[i]['points']
            for j in range(len(one_part)):
                rt_line.append(one_part[j])
        #
    # 
    line_data.append(LineString(rt_line))
    
    return all_stops,line_data

In [443]:
def get_rt_data(one_rt,rt_id):

    for_reestr = []

    stop_seq1,line_data1 = get_dir_data(one_rt[0]['features'],0,rt_id)

    stp_frw = one_rt[0]['properties']['ThreadMetaData']['EssentialStops'][0]['name']
    stp_bckw = one_rt[0]['properties']['ThreadMetaData']['EssentialStops'][1]['name']
    str_rt_nm = str(stp_frw) + "_" + str(stp_bckw)
    for_reestr.append(str_rt_nm)
#     length_frw=line_data[3].length
    length_frw=0
    length_bckw=0

    if len(one_rt) == 2:
        stop_seq2,line_data2 = get_dir_data(one_rt[1]['features'],1,rt_id)
        stop_seq = stop_seq1 + stop_seq2
        line_data = line_data1 + line_data2
        
        is_circle = False
#         length_bckw=line_data[3].length

    else:
        stop_seq = stop_seq1
        line_data = line_data1
        is_circle = True
#         length_bckw = 0

    # 
    
    for_reestr.append(is_circle)
    for_reestr.append(length_frw)
    for_reestr.append(length_bckw)
    return for_reestr, stop_seq, line_data

In [451]:
# dct_lnks = {}
big_reestr=[]
big_seq=[]
big_line=[]
i=0
# for i in range(len(lst_rts)):
for i in range(2):
    lnkid = lst_rts[i][0]
#     lnkid = lst_lineid[i]
    lnk_rt = 'https://yandex.ru/maps/{}/{}/routes/{}'.format(city_id, city_name, lnkid)
    js_rt = find_js(lnk_rt)
    one_rt=js_rt['masstransitLine']['features']
    rt_nbr = lst_rts[i][1]
    rt_tp = lst_rts[i][2]
    for_r_2 = [lnkid,rt_nbr,rt_tp]
#     need: rt_name,is_circle,length_frw,length_bckw
    for_reestr, stop_seq, line_data = get_rt_data(one_rt,lnkid)
    for_r_2 = for_r_2 + for_reestr
    big_reestr.append(for_r_2)
    big_seq.append(stop_seq)
    big_line.append(line_data)

#     dct_lnks[lnkid] = js_rt['masstransitLine']['features']
# 

In [458]:
big_seq[0][0]

['2193179075', 0, 1, 'stop__10002140', 'Овощная база', 53.253455, 50.2948622]

In [396]:
# line_length(line)
line = line_data[2]

# return length

In [402]:
crds = list(line.coords)

In [404]:
crds[0][0]

50.294937

In [405]:
from pyproj import Proj, transform

In [407]:
inProj = Proj('epsg:4326')
outProj = Proj('epsg:32637')

crds = list(line.coords)
lst_new_line=[]
for i in range(len(crds)):
    lat=crds[i][0]
    lon=crds[i][1]
    lat2,lon2 = transform(inProj,outProj,lat,lon)
    lst_new_line.append([lat2,lon2])
# 
new_line = LineString(lst_new_line)
length = new_line.length
print(length)

C:\Users\ochristm\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':


22484.173070391418
